In [3]:
import os

In [4]:
%pwd

'e:\\DL Projects\\Chicken-Disease-Classification\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'e:\\DL Projects\\Chicken-Disease-Classification'

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir : Path

In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath =  PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    
    

In [21]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r' ) as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion  = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2023-11-21 01:06:59,968: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-21 01:06:59,968: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-21 01:06:59,969: INFO: common: created directory at: artifacts]
[2023-11-21 01:06:59,969: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-21 01:07:04,932: INFO: 41612872: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "13bdec1469c44466b39cc7ade1e26bfa8b841ed42dcd3ee891035b00f2501090"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0A2A:159DDC:23741:2EBC7:655BAE58
Accept-Ranges: bytes
Date: Mon, 20 Nov 2023 19:07:05 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1262-QPG
X-Cache: MISS
